#Imports

In [ ]:
!pip install emoji==1.7
!pip install pyenchant
!pip install language_tool_python
!pip install pyspellchecker
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=d7c120cb45a89e757a6ce9ab17ce80669ee9bb070cdb1c02ef0e40fcc12f3071
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.4 MB/s eta 0:00

In [ ]:
import pandas as pd
from emoji import UNICODE_EMOJI
import re
from sentence_transformers import SentenceTransformer, util

#Uploading the Dataset


In [ ]:
train_url = "https://raw.githubusercontent.com/ArikRahman/Datathon/main/social-media-post-approval-prediction-with-marky%20(1)/train.csv"
test_url = "https://raw.githubusercontent.com/ArikRahman/Datathon/main/social-media-post-approval-prediction-with-marky%20(1)/test.csv"

df_train = pd.read_csv(train_url)
df_train2 = pd.read_csv(train_url)

df_test = pd.read_csv(test_url)

#Looking At Data

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   id                                    1645 non-null   object
 1   user_edited                           1645 non-null   bool  
 2   font                                  465 non-null    object
 3   image                                 1645 non-null   object
 4   caption                               1645 non-null   object
 5   parameters_chapter_title              1644 non-null   object
 6   parameters_chapter_summary            1641 non-null   object
 7   parameters_tone                       1588 non-null   object
 8   created_at                            1645 non-null   object
 9   user_id                               1645 non-null   object
 10  parameters_switchboard_template_name  1588 non-null   object
 11  parameters_theme              

In [ ]:
df_train.replace(df_train.iloc[0]["font"],1)

,id,user_edited,font,image,caption,parameters_chapter_title,parameters_chapter_summary,parameters_tone,created_at,user_id,parameters_switchboard_template_name,parameters_theme,parameters_prompt_template_name,parameters_photo_search_term,user_created_at,approved,has_logo
0,38c3f9e9-1b3c-4eaa-9ddf-57b477b20cf5,False,1,https://marky-image-posts.s3.amazonaws.com/0c6...,🌟 Break free from self-doubt! In fashion entre...,The Birth of Stout Baas,"Follow the journey of how Stout Baas, a self m...",promotional,2023-10-18T11:37:43.717704,7a591b49-1b59-4fa2-aeac-1dcc4f4cc92e,mega-template-3,joyful,reminder-limiting-belief,Stout Baas,2023-10-18T11:20:17.840709,False,True
1,0782787f-b214-49bc-a2c6-bbb77fbaf1c5,False,NaN,https://marky-image-posts.s3.amazonaws.com/568...,Express your unique style with our 10k Hip Hop...,Jewelry as a Form of Self-Expression,Delve into the role of jewelry as a form of se...,promotional,2023-10-04T15:25:12.986687,bbea9ffc-7666-4e2d-84a5-b6d2be200c2c,bnw-template-9,black & white,why-do-this,conveying a message,2023-10-04T15:06:57.689219,False,True
2,a7de4d69-bf31-456f-b853-d8ef0df319aa,False,NaN,https://marky-image-posts.s3.amazonaws.com/f46...,Ready to break free from the 9-5 grind? Discov...,Finding Your Muse,Uncover your true passions and discover how to...,informational,2023-09-14T23:30:55.445505,649226bcdd601e7d0df9a3b9,coffee-template-86,coffee,goal-advice,create a life of purpose,2023-08-12T15:58:09.271370,False,True
3,9070b324-32b7-4da5-92a7-4e923eab9103,False,NaN,https://marky-image-posts.s3.amazonaws.com/0fa...,"Introducing Rays Away, the game-changer in sun...",Targeting Eco-Conscious Individuals,Learn about the specific target audience of Ra...,inspirational,2023-09-21T00:42:29.909598,b270c878-2fbd-40e9-9b3c-dd77f9069611,bee-template-62-1,bee,did-you-know-2-cta,eco-conscious individuals,2023-09-15T14:34:06.259103,False,False
4,55a8b74b-09b6-4e7f-858c-d9bc44215a95,False,NaN,https://marky-image-posts.s3.amazonaws.com/e49...,Struggling with Twitter ghostwriting? 🤔 Don't ...,Overcoming Challenges and Staying Motivated,Navigate the challenges and obstacles that com...,informational,2023-10-16T09:43:10.988531,71db4a44-5826-4346-b561-db12c441a08e,tech-template-4,bold tech,stop-counterproductive-2,staying motivated in content creation,2023-10-16T09:35:53.001513,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,4ef0359d-42e2-423b-b224-a827acb166dd,False,NaN,https://marky-image-posts.s3.amazonaws.com/c4f...,📢 Last chance for sign enthusiasts! Explore ou...,signing,i sell products of sign,promotional,2023-10-19T11:53:19.138118,cbd37c27-35c2-4c80-b5f8-87d3875e29d1,new-template-27,ugc,its-not-too-late,sign equipment,2023-10-19T11:35:40.331223,False,False
6572,7cd1f278-a5fe-44a9-929d-b97bc4e19098,False,NaN,https://marky-image-posts.s3.us-east-2.amazona...,Join us at [dance school name] where age is ju...,Jazzing it Up,Immerse yourself in the energetic world of jaz...,informational,2023-09-04T07:28:33.647202,c6616693-55a8-4a21-b06b-e7aa59b8285b,pepe-meme-39,meme,solution-content-prompt,NaN,2023-09-04T06:45:36.620600,True,False
6573,995fe6ad-3972-43f6-8547-e0f4cad6134a,False,NaN,https://marky-image-posts.s3.amazonaws.com/dc9...,🌟 Struggling with fitness routine? No worries!...,Creating a Sustainable Health and Fitness Routine,Develop a personalized plan for long-term heal...,informational,2023-10-13T21:00:37.861120,cdfa87cd-4196-4cda-bcfd-45e04c397a91,bnw-template-2,black & white,dont-worry-2,long-term health and fitness,2023-10-13T20:42:12.566068,False,False
6574,7b1b6d75-da63-4580-96f2-1e4eb349af02,False,NaN,https://marky-image-posts.s3.amazonaws.com/791...,"Embrace diversity, expand horizons! Fostering ...",The Power of Cultural Understanding,Explore how cultural understanding can bridge ...,inspirational,2023-10-09T17:18:14.894363,fb831658-19f5-4af3-b927-7a58e638f742,new-template-26,ugc,better-did-you-know,cultural exchange,2023-10-09T17:15:22.327563,True,True


In [ ]:
df_train

,id,user_edited,font,image,caption,parameters_chapter_title,parameters_chapter_summary,parameters_tone,created_at,user_id,parameters_switchboard_template_name,parameters_theme,parameters_prompt_template_name,parameters_photo_search_term,user_created_at,approved,has_logo
0,38c3f9e9-1b3c-4eaa-9ddf-57b477b20cf5,False,https://marky-fonts.s3.us-east-2.amazonaws.com...,https://marky-image-posts.s3.amazonaws.com/0c6...,🌟 Break free from self-doubt! In fashion entre...,The Birth of Stout Baas,"Follow the journey of how Stout Baas, a self m...",promotional,2023-10-18T11:37:43.717704,7a591b49-1b59-4fa2-aeac-1dcc4f4cc92e,mega-template-3,joyful,reminder-limiting-belief,Stout Baas,2023-10-18T11:20:17.840709,False,True
1,0782787f-b214-49bc-a2c6-bbb77fbaf1c5,False,NaN,https://marky-image-posts.s3.amazonaws.com/568...,Express your unique style with our 10k Hip Hop...,Jewelry as a Form of Self-Expression,Delve into the role of jewelry as a form of se...,promotional,2023-10-04T15:25:12.986687,bbea9ffc-7666-4e2d-84a5-b6d2be200c2c,bnw-template-9,black & white,why-do-this,conveying a message,2023-10-04T15:06:57.689219,False,True
2,a7de4d69-bf31-456f-b853-d8ef0df319aa,False,NaN,https://marky-image-posts.s3.amazonaws.com/f46...,Ready to break free from the 9-5 grind? Discov...,Finding Your Muse,Uncover your true passions and discover how to...,informational,2023-09-14T23:30:55.445505,649226bcdd601e7d0df9a3b9,coffee-template-86,coffee,goal-advice,create a life of purpose,2023-08-12T15:58:09.271370,False,True
3,9070b324-32b7-4da5-92a7-4e923eab9103,False,NaN,https://marky-image-posts.s3.amazonaws.com/0fa...,"Introducing Rays Away, the game-changer in sun...",Targeting Eco-Conscious Individuals,Learn about the specific target audience of Ra...,inspirational,2023-09-21T00:42:29.909598,b270c878-2fbd-40e9-9b3c-dd77f9069611,bee-template-62-1,bee,did-you-know-2-cta,eco-conscious individuals,2023-09-15T14:34:06.259103,False,False
4,55a8b74b-09b6-4e7f-858c-d9bc44215a95,False,NaN,https://marky-image-posts.s3.amazonaws.com/e49...,Struggling with Twitter ghostwriting? 🤔 Don't ...,Overcoming Challenges and Staying Motivated,Navigate the challenges and obstacles that com...,informational,2023-10-16T09:43:10.988531,71db4a44-5826-4346-b561-db12c441a08e,tech-template-4,bold tech,stop-counterproductive-2,staying motivated in content creation,2023-10-16T09:35:53.001513,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,4ef0359d-42e2-423b-b224-a827acb166dd,False,NaN,https://marky-image-posts.s3.amazonaws.com/c4f...,📢 Last chance for sign enthusiasts! Explore ou...,signing,i sell products of sign,promotional,2023-10-19T11:53:19.138118,cbd37c27-35c2-4c80-b5f8-87d3875e29d1,new-template-27,ugc,its-not-too-late,sign equipment,2023-10-19T11:35:40.331223,False,False
6572,7cd1f278-a5fe-44a9-929d-b97bc4e19098,False,NaN,https://marky-image-posts.s3.us-east-2.amazona...,Join us at [dance school name] where age is ju...,Jazzing it Up,Immerse yourself in the energetic world of jaz...,informational,2023-09-04T07:28:33.647202,c6616693-55a8-4a21-b06b-e7aa59b8285b,pepe-meme-39,meme,solution-content-prompt,NaN,2023-09-04T06:45:36.620600,True,False
6573,995fe6ad-3972-43f6-8547-e0f4cad6134a,False,NaN,https://marky-image-posts.s3.amazonaws.com/dc9...,🌟 Struggling with fitness routine? No worries!...,Creating a Sustainable Health and Fitness Routine,Develop a personalized plan for long-term heal...,informational,2023-10-13T21:00:37.861120,cdfa87cd-4196-4cda-bcfd-45e04c397a91,bnw-template-2,black & white,dont-worry-2,long-term health and fitness,2023-10-13T20:42:12.566068,False,False
6574,7b1b6d75-da63-4580-96f2-1e4eb349af02,False,NaN,https://marky-image-posts.s3.amazonaws.com/791...,"Embrace diversity, expand horizons! Fostering ...",The Power of Cultural Understanding,Explore how cultural understanding can bridge ...,inspirational,2023-10-09T17:18:14.894363,fb831658-19f5-4af3-b927-7a58e638f742,new-template-26,ugc,better-did-you-know,cultural exchange,202

In [ ]:
value_counts = df_train['font'].value_counts()
print(value_counts)

https://marky-fonts.s3.us-east-2.amazonaws.com/Poppins-Regular.ttf            563
https://marky-fonts.s3.us-east-2.amazonaws.com/Oswald-Regular.ttf             269
https://marky-fonts.s3.us-east-2.amazonaws.com/PlayfairDisplay-Regular.ttf    258
https://marky-fonts.s3.us-east-2.amazonaws.com/Montserrat-Regular.ttf         217
https://marky-fonts.s3.us-east-2.amazonaws.com/Avenir+Regular.ttf             128
https://marky-fonts.s3.us-east-2.amazonaws.com/GreatVibes-Regular.ttf         106
https://marky-fonts.s3.us-east-2.amazonaws.com/Raleway-Regular.ttf             94
https://marky-fonts.s3.us-east-2.amazonaws.com/Roboto-Regular.ttf              82
https://marky-fonts.s3.us-east-2.amazonaws.com/Inter-Regular.ttf               68
https://marky-fonts.s3.us-east-2.amazonaws.com/Lato-Regular.ttf                52
https://marky-fonts.s3.us-east-2.amazonaws.com/Electrolize-Regular.ttf         51
https://marky-image-posts.s3.amazonaws.com/Ilisarniq-BlackItalic.ttf           43
https://marky-fo

In [ ]:
value_counts = df_train['user_edited'].value_counts()
print(value_counts)

False    6060
True      516
Name: user_edited, dtype: int64


In [ ]:
value_counts = df_train['parameters_tone'].value_counts()
print(value_counts)

informational    2155
promotional      2108
inspirational    2094
Name: parameters_tone, dtype: int64


In [ ]:
value_counts = df_train['parameters_theme'].value_counts()
print(value_counts)

meme             1241
blue              933
ugc               773
bee               620
teal              476
coffee            390
black & white     358
bold tech         326
Karate            295
fresh             280
minimalism        169
beauty            143
Mega              102
cyber              93
classic            51
Marky              17
john deere         12
membership          6
Active              4
joyful              1
UGC                 1
mega                1
Name: parameters_theme, dtype: int64


In [ ]:
value_counts = df_train['approved'].value_counts()
print(value_counts)

False    3887
True     2689
Name: approved, dtype: int64


In [ ]:
value_counts = df_train['has_logo'].value_counts()
print(value_counts)

False    3304
True     3272
Name: has_logo, dtype: int64


#Cleaning

In [ ]:
count = -1
for index, row in df_train.iterrows():
  count+=1
  if pd.isna(row['parameters_chapter_title']):
    df_train.at[index, 'parameters_chapter_title'] = " "
  if pd.isna(row['parameters_chapter_summary']):
    df_train.at[index, 'parameters_chapter_summary'] = " "

count = -1
for index, row in df_test.iterrows():
  count+=1
  if pd.isna(row['parameters_chapter_title']):
    df_test.at[index, 'parameters_chapter_title'] = " "
  if pd.isna(row['parameters_chapter_summary']):
    df_test.at[index, 'parameters_chapter_summary'] = " "


In [ ]:
# from spellchecker import SpellChecker
# from language_tool_python import LanguageTool

# def check_spelling_errors(text):
#     spell = SpellChecker()
#     spelling_errors = spell.unknown(text.split())
#     return spelling_errors

# def check_grammar_issues(text):
#      tool = LanguageTool('en-US')
#      grammar_issues = tool.check(text)
#      return grammar_issues

In [ ]:
# text = df_train.iloc[0]["caption"]
# grammar_issues = check_grammar_issues(text)
# list_errors = [];

# if grammar_issues:
#     print("Grammar issues found:")
#     for match in grammar_issues:
#         print(match)
#         list_errors.append(match)

In [ ]:
import nltk
from nltk.corpus import stopwords
def remove_numbers(text):
    pattern = r'\d+'
    return re.sub(pattern, '', text)

def remove_urls(text):
    return re.sub(r'https?://[^\s]+', '', text)

def remove_mentions(text):
    pattern = r'@[\w_]+'
    return re.sub(pattern, '', text)

def remove_punctuation(text):
    pattern = r'[!"#$\()*+-<=>@[\]^_`{|}~]'
    return re.sub(pattern, '', text)

def remove_emojis(text):
  result = ""
  for char in text:
    if char not in UNICODE_EMOJI['en'].keys():
      result += char
  return result

def remove_hashtags(text):
    # Removes hashtags and the words directly following them
    return re.sub(r'#\S+', '', text)

nltk.download('stopwords')  # Download the stopwords dataset
stop_words = set(stopwords.words('english'))  # Set of English stopwords

def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(filtered_words)  # Join the words back into a string

def remove_parentheses(text):
    return re.sub(r'\([^)]+\)', '', text)

def clean_text(text):
  text = remove_emojis(text)
  text = remove_urls(text)
  text = remove_parentheses(text)
  text = remove_numbers(text)
  text = remove_mentions(text)
  text = remove_hashtags(text)
  text = remove_punctuation(text)
  translator = str.maketrans('', '', "'")
  text = text.translate(translator)
  text = remove_stopwords(text)
  text = re.sub(r'\s+', ' ', text)
  text = text.lower()
  text = text.strip()
  return text


for i in range(df_train.shape[0]):
    text_caption = clean_text(df_train.loc[i, 'caption'])
    text_title = clean_text(df_train.loc[i, 'parameters_chapter_title'])
    text_summary = clean_text(df_train.loc[i, 'parameters_chapter_summary'])

    df_train['caption'][i] = text_caption
    df_train['parameters_chapter_title'][i] = text_title
    df_train['parameters_chapter_summary'][i] = text_summary

for i in range(df_test.shape[0]):
    text_caption = clean_text(df_test.loc[i, 'caption'])
    text_title = clean_text(df_test.loc[i, 'parameters_chapter_title'])
    text_summary = clean_text(df_test.loc[i, 'parameters_chapter_summary'])

    df_test['caption'][i] = text_caption
    df_test['parameters_chapter_title'][i] = text_title
    df_test['parameters_chapter_summary'][i] = text_summary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-175-791647603679>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['caption'][i] = text_caption
<ipython-input-175-791647603679>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['parameters_chapter_title'][i] = text_title
<ipython-input-175-791647603679>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [ ]:
count = -1
for index, row in df_train.iterrows():
    count += 1
    if pd.isna(row['font']):  # Check if 'font' is NA
        df_train.at[index, 'font'] = 0
    else:
        df_train.at[index, 'font'] = 1


In [ ]:
count = -1
for index, row in df_test.iterrows():
    count += 1
    if pd.isna(row['font']):  # Check if 'font' is NA
        df_test.at[index, 'font'] = 0
    else:
        df_test.at[index, 'font'] = 1


In [ ]:
count = -1
for index, row in df_train.iterrows():
  count+=1
  if pd.isna(row['parameters_tone']):
    df_train.at[index, 'parameters_tone'] = " "
  if pd.isna(row['parameters_switchboard_template_name']):
    df_train.at[index, 'parameters_switchboard_template_name'] = " "
  if pd.isna(row['parameters_theme']):
    df_train.at[index, 'parameters_theme'] = " "
  if pd.isna(row['parameters_prompt_template_name']):
    df_train.at[index, 'parameters_prompt_template_name'] = " "
  if pd.isna(row['parameters_photo_search_term']):
    df_train.at[index, 'parameters_photo_search_term'] = " "

count = -1
for index, row in df_test.iterrows():
  count+=1
  if pd.isna(row['parameters_tone']):
    df_test.at[index, 'parameters_tone'] = " "
  if pd.isna(row['parameters_switchboard_template_name']):
    df_test.at[index, 'parameters_switchboard_template_name'] = " "
  if pd.isna(row['parameters_theme']):
    df_test.at[index, 'parameters_theme'] = " "
  if pd.isna(row['parameters_prompt_template_name']):
    df_test.at[index, 'parameters_prompt_template_name'] = " "
  if pd.isna(row['parameters_photo_search_term']):
    df_test.at[index, 'parameters_photo_search_term'] = " "


In [ ]:
df_train['user_edited'] = df_train['user_edited'].map({True: 1, False: 0})
df_train['approved'] = df_train['approved'].map({True: 1, False: 0})
df_train['has_logo'] = df_train['has_logo'].map({True: 1, False: 0})

df_test['user_edited'] = df_test['user_edited'].map({True: 1, False: 0})
df_test['has_logo'] = df_test['has_logo'].map({True: 1, False: 0})

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

def lemm(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize each token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the lemmatized tokens back into text
    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text

# Apply the lemmatization to DataFrame columns
df_train['caption'] = df_train['caption'].apply(lambda x: lemm(x) if pd.notna(x) else x)
df_train['parameters_chapter_title'] = df_train['parameters_chapter_title'].apply(lambda x: lemm(x) if pd.notna(x) else x)
df_train['parameters_chapter_summary'] = df_train['parameters_chapter_summary'].apply(lambda x: lemm(x) if pd.notna(x) else x)

df_test['caption'] = df_test['caption'].apply(lambda x: lemm(x) if pd.notna(x) else x)
df_test['parameters_chapter_title'] = df_test['parameters_chapter_title'].apply(lambda x: lemm(x) if pd.notna(x) else x)
df_test['parameters_chapter_summary'] = df_test['parameters_chapter_summary'].apply(lambda x: lemm(x) if pd.notna(x) else x)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Cosine Similarity


In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
count = -1

for index, row in df_train.iterrows():
  count+=1
  sentence1 = df_train.at[index, "caption"]
  sentence2 = df_train.at[index, "parameters_chapter_summary"] + df_train.at[index, "parameters_theme"] + df_train.at[index, "parameters_chapter_title"]
# Encode the sentences into embeddings
  embeddings1 = model.encode(sentence1, convert_to_tensor=True)
  embeddings2 = model.encode(sentence2, convert_to_tensor=True)
  cosine_similarity1 = abs(util.pytorch_cos_sim(embeddings1, embeddings2))
  if(cosine_similarity1.item()>0.5):
    df_train.at[index, "Similarity"]  = 1
  else:
    df_train.at[index, "Similarity"]  = 0




In [ ]:
for index, row in df_test.iterrows():
  count+=1
  sentence1 = df_test.at[index, "caption"]
  sentence2 = df_test.at[index, "parameters_chapter_summary"] + df_test.at[index, "parameters_theme"] + df_test.at[index, "parameters_chapter_title"]
# Encode the sentences into embeddings
  embeddings1 = model.encode(sentence1, convert_to_tensor=True)
  embeddings2 = model.encode(sentence2, convert_to_tensor=True)
  cosine_similarity1 = abs(util.pytorch_cos_sim(embeddings1, embeddings2))
  if(cosine_similarity1.item()>0.5):
    df_test.at[index, "Similarity"]  = 1
  else:
    df_test.at[index, "Similarity"]  = 0

#Model

##Model Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import numpy as np

# Extract the text data from the "caption" column for training
text_data = df_train["caption"]

# Target variable
y = df_train["approved"]

# Extract and convert additional features to a float64 array for training
X_additional = df_train[['user_edited', 'font', 'has_logo', 'Similarity']].astype('float64').values

# Initialize and fit the TF-IDF vectorizer on the training data
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(text_data)

# Combine the TF-IDF matrix and additional features for training
X_combined = hstack([X_tfidf, X_additional])

# Create and train the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_combined, y)


X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.3, random_state=42)

# Make predictions and calculate accuracy on the test set
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Extract the text data from the "caption" column for testing (unseen data)
unseen_data = df_test["caption"]

# Extract and convert additional features to a float64 array for testing (unseen data)
X_unseen_additional = df_test[['user_edited', 'font', 'has_logo', 'Similarity']].astype('float64').values

# Transform the unseen text data using the same TF-IDF vectorizer
X_unseen_tfidf = tfidf_vectorizer.transform(unseen_data)  # Note the use of 'transform' instead of 'fit_transform'

# Combine the TF-IDF matrix and additional features for testing (unseen data)
X_unseen_combined = hstack([X_unseen_tfidf, X_unseen_additional])

# Make predictions on the unseen data
unseen_data_predictions = rf_classifier.predict(X_unseen_combined)


Accuracy: 1.0


##SVM Classifier

In [ ]:
from sklearn.svm import SVC  # Import Support Vector Classification model
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_combined, y)

y_pred_svm = svm_classifier.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy_svm}")
unseen_data_predictions_svm = svm_classifier.predict(X_unseen_combined)

Accuracy: 0.8388241256969082


##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression  # Import Logistic Regression model
logistic_classifier = LogisticRegression(random_state=42)
logistic_classifier.fit(X_combined, y)

y_pred_lr = logistic_classifier.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy: {accuracy_lr}")
unseen_data_predictions_lr = logistic_classifier.predict(X_unseen_combined)

Accuracy: 0.8043588443993918


##PasiveAggressive Classifier

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier  # Import PassiveAggressiveClassifier
pa_classifier = PassiveAggressiveClassifier(max_iter=1000, random_state=42)
pa_classifier.fit(X_combined, y)

y_pred_pa = pa_classifier.predict(X_test)
accuracy_pa = accuracy_score(y_test, y_pred_pa)
print(f"Accuracy: {accuracy_pa}")
unseen_data_predictions_pa = pa_classifier.predict(X_unseen_combined)

Accuracy: 1.0


##MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB  # Import Multinomial Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_combined, y)
y_pred_nbc = nb_classifier.predict(X_test)
accuracy_nbc = accuracy_score(y_test, y_pred_nbc)
print(f"Accuracy: {accuracy}")
unseen_data_predictions_nbc = nb_classifier.predict(X_unseen_combined)


Accuracy: 1.0


#Download with Proper Format

In [ ]:
count = 0
print("id, approved")
for prediction in unseen_data_predictions_svm:
  if(prediction == 0):
    prediction = "false"
  else:
    prediction = "true"
  print(df_test.iloc[count]["id"], prediction)
  count += 1

In [ ]:
import pandas as pd

# Assuming you have unseen_data_predictions and the corresponding ids
# Create a DataFrame with the 'id' and 'approved' columns
results_df = pd.DataFrame({'id': df_test['id'], 'approved': unseen_data_predictions_svm})

# Convert numeric labels to 'true' or 'false' strings
results_df['approved'] = results_df['approved'].map({0: 'false', 1: 'true'})

# Save the DataFrame to a CSV file
results_df.to_csv('predictions.csv', index=False)

# Provide the download link to the saved CSV file
from google.colab import files
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ADDITIONAL EXPLORATION

In [ ]:
df_train2

,id,user_edited,font,image,caption,parameters_chapter_title,parameters_chapter_summary,parameters_tone,created_at,user_id,parameters_switchboard_template_name,parameters_theme,parameters_prompt_template_name,parameters_photo_search_term,user_created_at,approved,has_logo
0,38c3f9e9-1b3c-4eaa-9ddf-57b477b20cf5,False,https://marky-fonts.s3.us-east-2.amazonaws.com...,https://marky-image-posts.s3.amazonaws.com/0c6...,🌟 Break free from self-doubt! In fashion entre...,The Birth of Stout Baas,"Follow the journey of how Stout Baas, a self m...",promotional,2023-10-18T11:37:43.717704,7a591b49-1b59-4fa2-aeac-1dcc4f4cc92e,mega-template-3,joyful,reminder-limiting-belief,Stout Baas,2023-10-18T11:20:17.840709,False,True
1,0782787f-b214-49bc-a2c6-bbb77fbaf1c5,False,NaN,https://marky-image-posts.s3.amazonaws.com/568...,Express your unique style with our 10k Hip Hop...,Jewelry as a Form of Self-Expression,Delve into the role of jewelry as a form of se...,promotional,2023-10-04T15:25:12.986687,bbea9ffc-7666-4e2d-84a5-b6d2be200c2c,bnw-template-9,black & white,why-do-this,conveying a message,2023-10-04T15:06:57.689219,False,True
2,a7de4d69-bf31-456f-b853-d8ef0df319aa,False,NaN,https://marky-image-posts.s3.amazonaws.com/f46...,Ready to break free from the 9-5 grind? Discov...,Finding Your Muse,Uncover your true passions and discover how to...,informational,2023-09-14T23:30:55.445505,649226bcdd601e7d0df9a3b9,coffee-template-86,coffee,goal-advice,create a life of purpose,2023-08-12T15:58:09.271370,False,True
3,9070b324-32b7-4da5-92a7-4e923eab9103,False,NaN,https://marky-image-posts.s3.amazonaws.com/0fa...,"Introducing Rays Away, the game-changer in sun...",Targeting Eco-Conscious Individuals,Learn about the specific target audience of Ra...,inspirational,2023-09-21T00:42:29.909598,b270c878-2fbd-40e9-9b3c-dd77f9069611,bee-template-62-1,bee,did-you-know-2-cta,eco-conscious individuals,2023-09-15T14:34:06.259103,False,False
4,55a8b74b-09b6-4e7f-858c-d9bc44215a95,False,NaN,https://marky-image-posts.s3.amazonaws.com/e49...,Struggling with Twitter ghostwriting? 🤔 Don't ...,Overcoming Challenges and Staying Motivated,Navigate the challenges and obstacles that com...,informational,2023-10-16T09:43:10.988531,71db4a44-5826-4346-b561-db12c441a08e,tech-template-4,bold tech,stop-counterproductive-2,staying motivated in content creation,2023-10-16T09:35:53.001513,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,4ef0359d-42e2-423b-b224-a827acb166dd,False,NaN,https://marky-image-posts.s3.amazonaws.com/c4f...,📢 Last chance for sign enthusiasts! Explore ou...,signing,i sell products of sign,promotional,2023-10-19T11:53:19.138118,cbd37c27-35c2-4c80-b5f8-87d3875e29d1,new-template-27,ugc,its-not-too-late,sign equipment,2023-10-19T11:35:40.331223,False,False
6572,7cd1f278-a5fe-44a9-929d-b97bc4e19098,False,NaN,https://marky-image-posts.s3.us-east-2.amazona...,Join us at [dance school name] where age is ju...,Jazzing it Up,Immerse yourself in the energetic world of jaz...,informational,2023-09-04T07:28:33.647202,c6616693-55a8-4a21-b06b-e7aa59b8285b,pepe-meme-39,meme,solution-content-prompt,NaN,2023-09-04T06:45:36.620600,True,False
6573,995fe6ad-3972-43f6-8547-e0f4cad6134a,False,NaN,https://marky-image-posts.s3.amazonaws.com/dc9...,🌟 Struggling with fitness routine? No worries!...,Creating a Sustainable Health and Fitness Routine,Develop a personalized plan for long-term heal...,informational,2023-10-13T21:00:37.861120,cdfa87cd-4196-4cda-bcfd-45e04c397a91,bnw-template-2,black & white,dont-worry-2,long-term health and fitness,2023-10-13T20:42:12.566068,False,False
6574,7b1b6d75-da63-4580-96f2-1e4eb349af02,False,NaN,https://marky-image-posts.s3.amazonaws.com/791...,"Embrace diversity, expand horizons! Fostering ...",The Power of Cultural Understanding,Explore how cultural understanding can bridge ...,inspirational,2023-10-09T17:18:14.894363,fb831658-19f5-4af3-b927-7a58e638f742,new-template-26,ugc,better-did-you-know,cultural exchange,202

In [ ]:
"""
id - Unique identifier for each post.
user_edited - A boolean value indicating if the user edited the post.
font - URL or reference to the font used in the post.
image - URL or reference to the associated image for the post.
caption - The content of the post, limited to 280 characters.
parameters_chapter_title - The title of the "talking point" we are creating a post for.
parameters_chapter_summary - The chapter summary from the PostParameters class.
parameters_tone - The tone of the post content.
created_at - Timestamp when the post was created.
user_id - Identifier for the user associated with the post.
parameters_switchboard_template_name - The name of the canvas we are using in this post (theres around 100 unique of these).
parameters_theme - Theme associated with the post.
parameters_prompt_template_name - The name of the prompt we use to create this post (theres around 100 unique of these).
parameters_photo_search_term - The search term used for the photo associated with the post.
user_created_at - Timestamp when the user was created.
approved - A boolean value indicating the post's approval status.
has_logo - A boolean value indicating if the post has a logo.
"""

totalApproved = 0
for i in range(len(df_train2)):
  row = df_train2.iloc[i]
  if row['approved']:
    totalApproved += 1

# USER EDITED AND APPROVED LINK
count=0
for i in range(len(df_train2)):
    row = df_train2.iloc[i]
    if row['approved'] and row['user_edited']:
      count+=1

print(count/totalApproved)

# HAS LOGO AND APPROVED LINK
count=0
for i in range(len(df_train2)):
    row = df_train2.iloc[i]
    if row['approved'] and row['has_logo']:
      count+=1

print(count/totalApproved)

# HAS LOGO AND APPROVED LINK
count1=0
count2 = 0
for i in range(len(df_train2)):
    row = df_train2.iloc[i]
    if row['user_edited'] and row['has_logo']:
      if row['approved']:
        count1+=1
      count2+=1

print(count1/totalApproved)
print(count1/count2)

0.1513573819263667
0.423205652658981
0.08404611379695054
0.740983606557377
